In [1]:
kalimat = []
kalimatAlign = []
PFADict = {}
PFASim = 0.9
w = 0.9
sudahAlign = []
goldAnnotation = []
precision = []
recall = []

punctuations = ['(','-lrb-',',','-','?','!',';','_',':','{','}','[','/',']','...','"','\'',')', '-rrb-',"'s",'"','ha',"'",'wa']


In [2]:
import os
from GlobalName import *
import re


class ReadFile:
    #read data format MSR
    def read_data(self,filename):
        global kalimatAlign
        bukaFile = open(os.path.join(filename), "r")
        for line in bukaFile.readlines():
            if line.split(None,1)[0] != '#' and line.split(None,1)[0] != "NULL" :
                x = line.rstrip('\n')

            if line.split(None,1)[0] == "NULL" :
                #dipisahkan menggunakan regular expression
                y = [x, re.sub('\s*(?:\(\{[^/]*/\s*/\s*}\)|NULL)\s*',' ', line)]
                kalimatAlign.append(y)

                #untuk mencari kata yang ada gold anotation/ gold standard
                goldAnno = re.findall('\s*(\(\{[^/]*/\s*/\s*}\))\s*',line)
                goldAnnotation.append(goldAnno)



In [3]:
import os
from GlobalName import *

In [4]:
import nltk
from nltk.tokenize import word_tokenize

In [5]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [6]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [7]:
class PreProcessing:

    #Open Data Stopwords Bahasa Indonesia
    data_stopword = "D:\EKSTENSI S1\semester 3\TA\CD TA Galih Rizky Prabowo\Code Program Galih Rizky Prabowo\Program_1\Resources\stopword.txt"
    file = open(data_stopword)
    stopwords = file.read().splitlines()

    def tokenisasi(self, sentences):
        result = []
        result = word_tokenize(sentences)
        return result

    def stemming(self, sentences):
        result = []
        factory = StemmerFactory()
        for i in range(len(sentences)):
            result.append(stemmer.stem(sentences[i]))
        return result

    def cleanGoldAnno(self):
        global goldAnnotation
        goldAnno = []
        tempo = []
        result= []
        for i in range(len(goldAnnotation)):
            goldAnno.append([])
            for j in range(len(goldAnnotation[i])):
                cek = re.findall('[^p][0-9][0-9]|[^p0-9][0-9]',goldAnnotation[i][j]) #dengan P dihitung->[0-9][0-9]|[0-9] #[0-9][0-9]|[0-9]|p[0-9][0-9]|p[0-9] #tanpa P -> [^p][0-9][0-9]|[^p0-9][0-9]
                for k in range (len(cek)):
                     cek[k] = cek[k].strip()
                goldAnno[i].append(cek)
            result.append(goldAnno)

        return result

    def LowerCase(self, sentence):
        result = [x.lower() for x in sentence]
        return  result

    def hapusNull(self):
        for i in range(len(goldAnnotation)):
            del goldAnnotation[i][0]






In [8]:
import random
from GlobalName import *


class classIdentical:

    def tambahAlign(self, alignResult, i):
        global sudahAlign
        sudahAlign.append([])
        for k in range(len(alignResult)):
            if alignResult[k] not in sudahAlign[i] :
                temp = [alignResult[k]]
                sudahAlign[i].extend(temp)
        return sudahAlign

    def alignIdenticalWords(self, kalimatDasar1, kalimatDasar2):
        identicalWords = []
        global sudahAlign
        t = 1
        #fungsi random untuk menghitung batas bawah
        #kanan = []
        #kiri = []
        for i in range(len(kalimatDasar1)):
            for j in range(len(kalimatDasar2)):
                if kalimatDasar1[i].lower() and kalimatDasar2[j].lower() not in punctuations:
                    if kalimatDasar1[i].lower() == kalimatDasar2[j].lower() and kalimatDasar2[j] not in punctuations:
                        identicalWords.append([j+1,t])
                        #kanan.append(t)
                        #kiri.append(j+1)
            t = t + 1

        #kanan.append(len(kalimat_pertama))
        #kiri.append(len(kalimat_kedua))

        #align tanda baca di akhir kalimat
        identicalWords.append([len(kalimatDasar2),len(kalimatDasar1)])
        #allList = []
        #for z in identicalWords:
            #allList = allList + z
        #k = 0
        #identicalWords2 = []
        #while k < len(identicalWords):
            #identicalWords2.append([random.choice(kiri),random.choice(kanan)])
            #k+=1
        #return identicalWords2
        return identicalWords



In [9]:
import PPDB
from PPDB import *
from GlobalName import *

class classPFA:

    def tambahAlignPFA(self, alignResult, i, cekidentical):
        global sudahAlign
        sudahAlign.append([])
        tempTidakAda = []
        tempKata = []
        for l in range(len(cekidentical)):
            tempKata.append(cekidentical[l][0])
        for j in range(len(alignResult)):
            if alignResult[j][0] not in tempKata:
                tempTidakAda.append(alignResult[j])
        for k in range(len(tempTidakAda)):
            if tempTidakAda[k] not in sudahAlign[i]:
                temp = [tempTidakAda[k]]
                sudahAlign[i].extend(temp)
        return sudahAlign


    def alignWordSimilarity(self, kalimatDasar1, kalimatDasar2):
        global sudahAlign
        wordsSimilarity = []
        t = 1
        #fungsi random untuk menghitung batas bawah
        #kanan = []
        #kiri = []
        for i in range(len(kalimatDasar1)):
            for j in range(len(kalimatDasar2)):
                if kalimatDasar1[i].lower() and kalimatDasar2[j].lower() not in punctuations:
                    if PPDB.presentInPFA(kalimatDasar1[i], kalimatDasar2[j]) is True:
                        wordsSimilarity.append([j + 1, t])
                        #kanan.append(t)
                        #kiri.append(j + 1)
            t += 1
        #kanan.append(len(kalimatDasar1))
        #kiri.append(len(kalimatDasar2))

        res = []
        for m in range(len(wordsSimilarity)):
            if wordsSimilarity[m] not in res:
                res.append(wordsSimilarity[m])

        #allList = []
        #for v in wordsSimilarity:
            #allList = allList + v
        #k = 0
        #wordSim2 = []
        #while k < len(wordsSimilarity):
            #wordSim2.append([random.choice(kiri), random.choice(kanan)])
            #k += 1
        #return wordSim2
        return res





In [10]:
from GlobalName import *
from Preprocessing import *
import random

class classWordSequence:

    def tambahAlign(self, alignResult, i):
        global sudahAlign
        sudahAlign.append([])
        for k in range(len(alignResult)):
            if alignResult[k] not in sudahAlign[i] :
                temp = [alignResult[k]]
                sudahAlign[i].extend(temp)
        return sudahAlign

    def isSublist(self, A, B):
        # returns True if A is a sublist of B, False otherwise
        sub = True

        for item in A:
            if item not in B:
                sub = False
                break
        return sub

    # ====================================================================#
    def findAllCommonContiguousSublists(self,A, B, turnToLowerCases=True):
        a = []
        b = []
        for item in A:
            a.append(item)
        for item in B:
            b.append(item)

        if turnToLowerCases:
            for i in range(len(a)):
                a[i] = a[i].lower()
            for i in range(len(b)):
                b[i] = b[i].lower()
        commonContiguousSublists = []
        swapped = False
        if len(a) > len(b):
            temp = a
            a = b
            b = temp
            swapped = True
        maxSize = len(a)
        for size in range(maxSize, 0, -1):
            startingIndicesForA = [item for item in range(0, len(a)-size+1)]
            startingIndicesForB = [item for item in range(0, len(b)-size+1)]
            for i in startingIndicesForA:
                for j in startingIndicesForB:
                    if a[i:i+size] == b[j:j+size]:
                        # check if a contiguous superset has already been inserted; don't insert this one in that case
                        alreadyInserted = False
                        currentAIndices = [item for item in range(i,i+size)]
                        currentBIndices = [item for item in range(j,j+size)]
                        for item in commonContiguousSublists:
                            if classWordSequence().isSublist(currentAIndices, item[0]) and classWordSequence().isSublist(currentBIndices, item[1]):
                                alreadyInserted = True
                                break
                        if not alreadyInserted:
                            commonContiguousSublists.append([currentAIndices, currentBIndices])

        if swapped:
            for item in commonContiguousSublists:
                temp = item[0]
                item[0] = item[1]
                item[1] = temp
        return commonContiguousSublists

    # ====================================================================#
    #align word sequences
    def alignSequences(self,kalimatDasar1,kalimatDasar2):
        alignments = []
        #fungsi random untuk menghitung batas bawah
        #rlist = []
        #llist = []
        sourceWordIndicesAlreadyAligned= []
        targetWordIndicesAlreadyAligned= []
    # align all (>=2)-gram matches with at least one content word
        commonContiguousSublists  = classWordSequence().findAllCommonContiguousSublists(kalimatDasar1, kalimatDasar2, True)
        for item in commonContiguousSublists:
                allStopWords = True
                for jtem in item:
                    if jtem not in PreProcessing.stopwords and jtem not in punctuations:
                        allStopWords = False
                        break
                if len(item[0]) >= 2 and not allStopWords:
                    for j in range(len(item[0])):
                        if item[0][j]+1 not in sourceWordIndicesAlreadyAligned and item[1][j]+1 not in targetWordIndicesAlreadyAligned and [item[0][j]+1, item[1][j]+1] not in alignments:
                            alignments.append([item[1][j]+1, item[0][j]+1])
                            sourceWordIndicesAlreadyAligned.append(item[0][j]+1)
                            targetWordIndicesAlreadyAligned.append(item[1][j]+1)
                            #rlist.append(item[0][j]+1)
                            #llist.append(item[1][j]+1)

        #rlist.append(len(sourceWordIndicesAlreadyAligned))
        #llist.append(len(targetWordIndicesAlreadyAligned))

        #semuaList = []
        #for b in alignments:
            #semuaList = semuaList + b
        #m = 0
        #alignments2 = []
        #while m < len(alignments):
            #alignments2.append([random.choice(llist), random.choice(rlist)])
            #m+= 1
        #return alignments2
        return alignments
    # ====================================================================#


In [11]:
from __future__ import division
from Preprocessing import *

class Evaluasi:

    def precision(self,jumlahBenar,jumlahDariSistem):
        try:
            precision = jumlahBenar/jumlahDariSistem
        except ZeroDivisionError:
            precision = 0
        return round(precision,2)

    def recall(self,jumlahBenar,goldAnnotation):
        recall = jumlahBenar/goldAnnotation
        return round(recall,2)

    def F1Measure(self,precision,recall):
        if precision and recall != 0:
            F1 = 2*precision*recall/(precision+recall)
        else:
            F1=0
        return round(F1,2)

    def Rata_rata(self,average):
        result = sum(average)/len(average)
        return round(result,4)

    def jumlahBenar(self,sudahAlign,kalimat_kedua,listAnnotation):
        true = 0
        for i in range(len(sudahAlign)):
            for j in range(len(kalimat_kedua)):
                if sudahAlign[i][0] == kalimat_kedua[j]:
                    listAnno = PreProcessing().LowerCase(listAnnotation[j])
                    if str(sudahAlign[i][1]) in listAnno:
                            true += 1
        return true
    def jumlahGoldAnnotation(self, goldAnnotation):
        a = 0
        for i in range(len(goldAnnotation)):
            if not goldAnnotation[i]:
                continue
            else:
                a += len(goldAnnotation[i])
        return a

    def jumlahSistem(self, sudahAlign):
        true = len(sudahAlign)
        return true

    def printAnnotation(self,GA,kalimatDasar2):
        temp = []
        for i in range (len(kalimatDasar2)):
            temp.append([GA[i], kalimatDasar2[i]])
        return temp

    @staticmethod
    def transformResult(kalimatDasar2, sudahAlign):
        result = []
        for x in range(len(sudahAlign)):
            result.append([kalimatDasar2[sudahAlign[x][0] - 1], sudahAlign[x][1]])
        y = []
        for i in result:
            if i not in y:
                y.append(i)
        return y



In [12]:
import os
from GlobalName import *
import nltk
from nltk.tokenize import word_tokenize
import re
from Preprocessing import *
from ReadFile import *
from Evaluasi import *
#####Aligner fitur monolingual word alignment######
from AlignIdentical import *
from AlignPFA import *
from AlignWordSequence import *
###################################################
import random


In [13]:
global kalimat
global sudahAlign
Rata_rata_precision = []
Rata_rata_recall = []
Rata_rata_F1 = []

In [14]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Amalia Asti
[nltk_data]     H\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
class mainProgram:
    def runProgram(self,filename):
        ReadFile().read_data(filename)
        PreProcessing().hapusNull()
        listAnnotation = PreProcessing().cleanGoldAnno()
        PPDB.loadPFA()
        kalimat.extend(kalimatAlign)

        write_precision = open('precision1.txt', 'w')
        write_recall = open('recall1.txt', 'w')
        write_F1 = open('F11.txt', 'w')
        write_corpus = open('resultCorpus1.txt', 'w')

        #konter = 0

        for i in range(len(kalimat)):
            #Preprocessing kalimat inputan
            kalimat_pertama = kalimat[i][0]
            kalimat_kedua = kalimat[i][1]
            kalimatToken1 = PreProcessing().tokenisasi(kalimat_pertama)
            kalimatToken2 = PreProcessing().tokenisasi(kalimat_kedua)
            kalimatDasar1 = PreProcessing().stemming(kalimatToken1)
            kalimatDasar2 = PreProcessing().stemming(kalimatToken2)

            print ("----------------------------------------")
            print ("Pasangan Ayat ke %d\r" % (i+1))
            print ("Kalimat pertama  :" ,kalimatToken1)
            print ("Kalimat kedua    :" ,kalimatToken2)
            print ("Gold Standard  :", Evaluasi().printAnnotation(kalimatDasar2, goldAnnotation[i]))
            GA = Evaluasi().printAnnotation(kalimatDasar2, goldAnnotation[i])
            print ("----------------------------------------")
            print ("#########################################")

            #Identifikasi fitur align identical words
            cekidentical = classIdentical().alignIdenticalWords(kalimatDasar1, kalimatDasar2)
            classIdentical().tambahAlign(cekidentical, i)
            print("Identical Words      :", cekidentical)
            #==============================================================================#
            
            #identifikasi GAP atau jarak untuk membangun parafrase secara otomatis
            file_content = open('parafrase.txt').readlines()
            con = 0
            global sudahAlign
            while (con < len(cekidentical)):
                if con+1 < len(cekidentical):
                    #print (cekidentical[con + 1][0] - cekidentical[con][0])
                    if ((cekidentical[con+1][0] - cekidentical[con][0]) >= 2) and ((cekidentical[con+1][1] - cekidentical[con][1]) >= 2):
                        kanan = []
                        kiri = []
                        kanankata = []
                        kirikata = []
                        parafrase = []
                        parafrasekata = []
                        isikiri = (cekidentical[con+1][0] - cekidentical[con][0])
                        isikanan = (cekidentical[con+1][1] - cekidentical[con][1])
                        while isikiri > 1:
                            isikiri = isikiri - 1
                            liskiri = cekidentical[con][0] + isikiri
                            #print liskiri
                            kiri.append(liskiri)
                        while isikanan > 1:
                            isikanan = isikanan - 1
                            liskanan = cekidentical[con][1] + isikanan
                            #print liskanan
                            kanan.append(liskanan)
                        #for m in kiri:
                            #kirikata.append(kalimatToken2[m - 1])
                        #for n in kanan:
                            #kanankata.append(kalimatToken1[n - 1])
                        parafrase.append([kanan,kiri])
                        #parafrasekata.append([kanankata,kirikata])
                        #if (len(parafrasekata) > 0):
                            #print file_content[konter]
                            #print konter
                            #konter+=1
                        print ("Paraphrase Otomatis   :", parafrase)
#                         print ("Paraphrase Otomatis kata   :", parafrasekata)
                con+=1

            #cekidentical2 = classIdentical().alignIdenticalWords2(kalimatDasar1, kalimatDasar2)
            #classIdentical().insertAlign(cekidentical, i)
            #print "Identical Words      :", cekidentical2
            # ==============================================================================#

            #Identifikasi fitur align PPDB
            cekSimilar = classPFA().alignWordSimilarity(kalimatDasar1, kalimatDasar2)
            classPFA().tambahAlignPFA(cekSimilar, i, cekidentical)
            print ("Parafrase Kata       :", cekSimilar)

            #Identifikasi fitur align word sequences
            cekSequences = classWordSequence().alignSequences(kalimatDasar1, kalimatDasar2)
            classWordSequence().tambahAlign(cekSequences, i)
            print ("Word Sequence        :",cekSequences)

            "#########################################"
            "Hasil alignment"
            print( "----------------------------------------")
            hasilMSR = Evaluasi.transformResult(kalimatDasar2, sudahAlign[i])

            # Memanggil method hitung jumlah benar, jumlah gold standard dan jumlah sistem dari class evaluasi
            print ("----------------------------------------")
            jumlahBenar = Evaluasi().jumlahBenar(hasilMSR, kalimatDasar2, listAnnotation[i][i])
            print ("Jumlah Benar", jumlahBenar)

            jumlahSistem = Evaluasi().jumlahSistem(hasilMSR)
            print ("Jumlah Sistem", jumlahSistem)

            jumlahGoldAnnotation = Evaluasi().jumlahGoldAnnotation(listAnnotation[i][i])
            print ("jumlah Gold Annotation", jumlahGoldAnnotation)
            print ("----------------------------------------")
            print ("----------------------------------------")

            #Menghitung hasil precision, recall dan F1 tiap pasangan kata dan dirata
            precision = Evaluasi().precision(jumlahBenar, jumlahSistem)
            Rata_rata_precision.append(precision)
            write_precision.write(str(precision) + '\n')
            print ("precision", precision)


            recall = Evaluasi().recall(jumlahBenar, jumlahGoldAnnotation)
            Rata_rata_recall.append(recall)
            write_recall.write(str(recall) + '\n')
            print ("recall", recall)


            F1 = Evaluasi().F1Measure(precision, recall)
            Rata_rata_F1.append(F1)
            write_F1.write(str(F1) + '\n')
            print ("F1", F1)

            #Menyimpan kalimat yang telah di-align dalam bentuk .txt
            write_corpus.write("# sentence pair %d\r\n" % (i+1))
            write_corpus.write(str(kalimat_pertama) + '\n')
            write_corpus.write(str(["NULL", "({ / / })", GA]) + '\n')


        # Menyimpan file dalam bentuk txt
        write_precision.close()
        write_recall.close()
        write_F1.close()
        write_corpus.close()


        print ("*************Hasil Rata - Rata Akhir*************")

        RataPrecision = Evaluasi().Rata_rata(Rata_rata_precision)
        print ("Rata-rata Precision   :",RataPrecision)

        RataRecall = Evaluasi().Rata_rata(Rata_rata_recall)
        print ("Rata-rata Recall      :", RataRecall)

        RataF1 = Evaluasi().Rata_rata(Rata_rata_F1)
        print ("F1measure total       :", RataF1)
        
mainProgram().runProgram('data_coba.txt')


----------------------------------------
Pasangan Ayat ke 1
Kalimat pertama  : ['Dengan', 'nama', 'Allah', 'Pemberi', 'Kasih', 'Yang', 'Maha', 'Pengasih', '.']
Kalimat kedua    : ['Dengan', 'menyebut', 'nama', 'Allah', 'Yang', 'Maha', 'Pemurah', 'lagi', 'Maha', 'Penyayang', '.']
Gold Standard  : [['dengan', '({ 1 / / })'], ['sebut', '({ / / })'], ['nama', '({ 2 / / })'], ['allah', '({ 3 / / })'], ['yang', '({ 6 / / })'], ['maha', '({ 7 / / })'], ['murah', '({ 9 / / })'], ['lagi', '({ / / })'], ['maha', '({ 7 / / })'], ['sayang', '({  / / })'], ['', '({ 10 / / })']]
----------------------------------------
#########################################
Identical Words      : [[1, 1], [3, 2], [4, 3], [5, 6], [6, 7], [9, 7], [11, 9]]
Paraphrase Otomatis   : [[[8], [10]]]
Parafrase Kata       : []
Word Sequence        : [[3, 2], [4, 3], [5, 6], [6, 7]]
----------------------------------------
----------------------------------------
Jumlah Benar 6
Jumlah Sistem 6
jumlah Gold Annotation 8
------

----------------------------------------
Pasangan Ayat ke 8
Kalimat pertama  : ['Kamu', 'sekali-kali', 'tidak', 'sampai', 'kepada', 'kebajikan', 'yang', 'sempurna', 'sebelum', 'kamu', 'menafkahkan', 'sebahagian', 'harta', 'yang', 'kamu', 'cintai', 'dan', 'apa', 'saja', 'yang', 'kamu', 'nafkahkan', 'maka', 'sesungguhnya', 'Allah', 'mengetahuinya', '.']
Kalimat kedua    : ['kamu', 'sekali-kali', 'tidak', 'akan', 'meraih', 'kebajikan', 'yang', 'disempurnakan', 'sebelum', 'sssskamu', 'menafkahkan', 'sebagian', 'apa', 'yang', 'kamu', 'sukai', 'Dan', 'apa', 'saja', 'yang', 'kamu', 'nafkahkan', 'maka', 'seseungguhnya', 'Allah', 'Maha', 'mengetahui', 'tentang', 'hal', 'itu', '.']
Gold Standard  : [['kamu', '({ 1 / / })'], ['sekali-kali', '({ 2 / / })'], ['tidak', '({ 3 / / })'], ['akan', '({ 4 / / })'], ['raih', '({ / / })'], ['kebaji', '({ 6 / / })'], ['yang', '({ / / })'], ['sempurna', '({ 7 / / })'], ['belum', '({ 8 / / })'], ['sssskamu', '({ 9 / / })'], ['nafkah', '({ 10 / / })'], ['bagi',

Paraphrase Otomatis   : [[[4, 3], [7, 6]]]
Paraphrase Otomatis   : [[[17], [21]]]
Parafrase Kata       : []
Word Sequence        : [[11, 7], [12, 8], [13, 9], [14, 10], [15, 11], [16, 12], [17, 13], [18, 14], [19, 15], [20, 16], [8, 5], [9, 6]]
----------------------------------------
----------------------------------------
Jumlah Benar 15
Jumlah Sistem 15
jumlah Gold Annotation 18
----------------------------------------
----------------------------------------
precision 1.0
recall 0.83
F1 0.91
----------------------------------------
Pasangan Ayat ke 13
Kalimat pertama  : ['Hai', 'orang-orang', 'yang', 'beriman', 'jika', 'kamu', 'mengikuti', 'sebahagian', 'dari', 'orang-orang', 'yang', 'diberi', 'Al', 'Kitab', 'niscaya', 'mereka', 'akan', 'mengembalikan', 'kamu', 'menjadi', 'orang', 'kafir', 'sesudah', 'kamu', 'beriman', '.']
Kalimat kedua    : ['Hai', 'orang-orang', 'yang', 'beriman', 'Jika', 'kamu', 'mengikuti', 'segolongan', 'dari', 'orang-orang', 'yang', 'diberi', 'Al-kitab', 'n

----------------------------------------
Pasangan Ayat ke 19
Kalimat pertama  : ['Kamu', 'adalah', 'umat', 'yang', 'terbaik', 'yang', 'dilahirkan', 'untuk', 'manusia', ',', 'menyuruh', 'kepada', 'yang', "ma'ruf", ',', 'dan', 'mencegah', 'dari', 'yang', 'munkar', ',', 'dan', 'beriman', 'kepada', 'Allah', '.', 'Sekiranya', 'Ahli', 'Kitab', 'beriman', ',', 'tentulah', 'itu', 'lebih', 'baik', 'bagi', 'mereka', ',', 'di', 'antara', 'mereka', 'ada', 'yang', 'beriman', ',', 'dan', 'kebanyakan', 'mereka', 'adalah', 'orang-orang', 'yang', 'fasik', '.']
Kalimat kedua    : ['Kamu', 'Kaum', 'Muslim', 'adalah', 'umat', 'yang', 'terbaik', 'yang', 'dikeluarkan', 'untuk', 'manusia', 'menyuruh', 'kepada', 'yang', "ma'ruf", 'dan', 'mencegah', 'yang', 'munkar', 'dan', 'beriman', 'kepada', 'Allah', 'Dan', 'jika', 'seandianya', 'Ahl', 'Al-kitab', 'beriman', 'tentulah', 'itu', 'lebih', 'baik', 'bagi', 'mereka', 'diantara', 'mereka', 'ada', 'orang-orang', 'muknim', 'yang', 'mantap', 'imannya', 'dan', 'kebany

----------------------------------------
Pasangan Ayat ke 22
Kalimat pertama  : ['Dan', 'apa', 'saja', 'kebajikan', 'yang', 'mereka', 'kerjakan', ',', 'maka', 'sekali-kali', 'mereka', 'tidak', 'dihalangi', 'menenerima', 'pahalanya', 'dan', 'Allah', 'Maha', 'Mengetahui', 'orang-orang', 'yang', 'bertakwa', '.']
Kalimat kedua    : ['dan', 'apa', 'saja', 'kebajikan', 'yang', 'mereka', 'kerjakan', 'maka', 'sekali-kali', 'mereka', 'tidak', 'dikufuri', 'senantiasa', 'disyukuri', 'dan', 'diberi', 'ganjaran', 'dan', 'Allah', 'Maha', 'Mengetahui', 'orang-orang', 'yang', 'bertakwa', '.']
Gold Standard  : [['dan', '({ 1 15 / / })'], ['apa', '({ 2 / / })'], ['saja', '({ 3 / / })'], ['kebaji', '({ 4 / / })'], ['yang', '({ 5 / / })'], ['mereka', '({ 6 / / })'], ['kerja', '({ 7 / / })'], ['maka', '({ 8 / / })'], ['sekali-kali', '({ 9 / / })'], ['mereka', '({ 10 / / })'], ['tidak', '({ 11 / / })'], ['kufur', '({ / / })'], ['senantiasa', '({ / / })'], ['syukur', '({ / / })'], ['dan', '({ 1 15 / / })'], 

----------------------------------------
Pasangan Ayat ke 26
Kalimat pertama  : ['Sungguh', 'Allah', 'telah', 'menolong', 'kamu', 'dalam', 'peperangan', 'Badar', 'padahal', 'kamu', 'adalah', 'ketika', 'itu', 'orang-orang', 'yang', 'lemah', 'Karena', 'itu', 'bertakwalah', 'kepada', 'Allah', 'supaya', 'kamu', 'mensyukuriNya', '.']
Kalimat kedua    : ['dan', 'demi', 'Allah', 'Sungguh', 'Allah', 'telah', 'menolong', 'kamu', 'dalam', 'perang', 'Badar', 'sedangkan', 'kamu', 'ketika', 'itu', 'adalah', 'orang-orang', 'yang', 'lemah', 'Karena', 'itu', 'bertawakallah', 'kepada', 'Allah', 'supaya', 'kamu', 'mensyukuriNya', '.']
Gold Standard  : [['dan', '({ / / })'], ['demi', '({ / / })'], ['allah', '({ 2 / / })'], ['sungguh', '({ 1 / / })'], ['allah', '({ 2 / / })'], ['telah', '({ 3 / / })'], ['tolong', '({ 4 / / })'], ['kamu', '({ 5 / / })'], ['dalam', '({ 6 / / })'], ['perang', '({ 7 / / })'], ['badar', '({ 8 / / })'], ['sedang', '({ / / })'], ['kamu', '({ 10 / / })'], ['ketika', '({ 11 / / })

----------------------------------------
Pasangan Ayat ke 32
Kalimat pertama  : ['Kepunyaan', 'Allah', 'apa', 'yang', 'ada', 'dilangit', 'dan', 'yang', 'ada', 'dibumi', 'Dia', 'memberi', 'ampun', 'kepada', 'siapa', 'yang', 'Dia', 'kehendaki', 'Dia', 'menyiksa', 'siapa', 'yang', 'Dia', 'kehendaki', 'dan', 'Allah', 'Maha', 'Pengampun', 'lagi', 'Maha', 'Penyayang', '.']
Kalimat kedua    : ['dan', 'mllik', 'Allah', 'apa', 'yang', 'dilangit', 'dan', 'apa', 'yang', 'dibumi', 'Dia', 'memberi', 'ampun', 'kepada', 'siapa', 'yang', 'Dia', 'kehendaki', 'dan', 'menyiksa', 'siapa', 'yang', 'Dia', 'kehendaki', 'berdasarkan', 'kemauan', 'dan', 'kecenderungan', 'masing-masing', 'dan', 'Allah', 'Maha', 'Pengampun', 'lagi', 'Maha', 'Pengasih', '.']
Gold Standard  : [['dan', '({ / / })'], ['mllik', '({ / / })'], ['allah', '({ 2 / / })'], ['apa', '({ 3 / / })'], ['yang', '({ 4 / / })'], ['langit', '({ 6 / / })'], ['dan', '({ 7 / / })'], ['apa', '({ 9 / / })'], ['yang', '({ 8 / / })'], ['bumi', '({ 10 / / 

Paraphrase Otomatis   : [[[19], [20]]]
Parafrase Kata       : [[20, 4], [20, 9], [20, 19]]
Word Sequence        : [[15, 14], [16, 15], [17, 16], [18, 17], [19, 18], [21, 20], [22, 21], [23, 22], [24, 23], [1, 1], [2, 2], [3, 3], [5, 5], [6, 6], [7, 7], [11, 10], [12, 11]]
----------------------------------------
----------------------------------------
Jumlah Benar 20
Jumlah Sistem 21
jumlah Gold Annotation 20
----------------------------------------
----------------------------------------
precision 0.95
recall 1.0
F1 0.97
----------------------------------------
Pasangan Ayat ke 37
Kalimat pertama  : ['yaitu', 'orang-orang', 'yang', 'menafkahkan', 'hartanya', 'baik', 'diwaktu', 'lapang', 'maupun', 'sempit', 'dan', 'orang-orang', 'yang', 'menahan', 'amarahnya', 'dan', 'memaafkan', 'kesalahan', 'orang', 'Allah', 'menyukai', 'orang-orang', 'yang', 'berbuat', 'kebajikan', '.']
Kalimat kedua    : ['Yaitu', 'orang-orang', 'yang', 'menafkahkan', 'hartanya', 'diwaktu', 'lapang', 'maupun', 's

----------------------------------------
Pasangan Ayat ke 40
Kalimat pertama  : ['Sesungguhnya', 'telah', 'berlalu', 'sebelum', 'kamu', 'sunnah-sunnah', 'Allah', 'Karena', 'itu', 'berjalanlah', 'kamu', 'di', 'muka', 'bumi', 'dan', 'perhatikanlah', 'bagaimana', 'akibat', 'orang-orang', 'yang', 'mendustakan', 'rasul-rasul', '.']
Kalimat kedua    : ['Sungguh', 'telah', 'berlalu', 'sebelum', 'kamu', 'sunah-sunah', 'karena', 'itu', 'berjalanlah', 'di', 'bumi', 'perhatikanlah', 'bagaimana', 'kesudahan', 'para', 'pendusta', '.']
Gold Standard  : [['sungguh', '({ 1 / / })'], ['telah', '({ 2 / / })'], ['lalu', '({ 3 / / })'], ['belum', '({ 4 / / })'], ['kamu', '({ 5 / / })'], ['sunah', '({ 6 / / })'], ['karena', '({ 8 / / })'], ['itu', '({ 9 / / })'], ['jalan', '({ 10 / / })'], ['di', '({ / / })'], ['bumi', '({ 14 / / })'], ['perhati', '({ 16 / / })'], ['bagaimana', '({ 17 / / })'], ['sudah', '({ / / })'], ['para', '({ / / })'], ['dusta', '({ 21 / / })'], ['', '({ 23 / / })']]
-----------------